# Model Validation Instructions

Before we can use the `ModelValidator` class, we need to make sure we can access the `pyCIMS` module. Since we are still developing the module, we can't install it from PyPI. Instead, we will make sure the module is in our PATH. The code below assumes you started the jupyter notebook from inside the `pycims_prototype/code/model_validation` directory. 

In [8]:
import os
import sys
import pprint as pp

sys.path.insert(0, os.path.abspath('../../')) # Assumes ipynb ran from pycims_prototype/code/model_validation
sys.path.insert(0, os.path.abspath('../../pyCIMS')) # Assumes ipynb ran from pycims_prototype/code/model_validation

import pyCIMS

Now that we have loaded the `ModelValidator` from the `pyCIMS` module we can start to use it. First we will instantiate the `ModelValidator` class. To instantiate the class we must provide the location of the excel file specifying the model description.

Optionally you can also provide a `node_col` parameter. This tells the model validator what the name of column specifying Node names. In the current model description (2019-12-16) this column is `"Node"`. If not provided with a value, this parameter defaults to `"Node"`. 

In [6]:
model_description_file = '../../pyCIMS_model_description.xlsm' # Assumes ipynb ran from pycims_prototype/code/model_validation

model_validator = pyCIMS.ModelValidator(model_description_file)

Next, using our `model_validator` we can use the `validate()` method to check for any errors in our model description. There are a couple of paramters for this method. I'll explain them below: 
* **`verbose`** : Determines whether the method will use print statements to notify of any problems identified in the model description. Here we have set verbose to be True so that we will see printed statements letting us know about the errors. . 

* **`raise_warnings`** : Determines whether the method will raise warnings when it identifies problems in the model description. Warnings are more "in your face" than print statements, appearing in red for the user. However, warnings do go away if you run the cell multiple times. Here, we have set raise_warnings to False. We will just look at the printed statements and the resulting dictionary (next cell)


In [7]:
model_validator.validate(verbose=True, raise_warnings=False)

2 node name/branch mismatches. See ModelValidator.warnings['mismatched_node_names'] for more info
3 references to unspecified nodes. See ModelValidator.warnings['unspecified_nodes'] for more info
1 non-root nodes are never referenced. See ModelValidator.warnings['unreferenced_nodes'] for more info
1 nodes were specified but don't provide a service. See ModelValidator.warnings['nodes_no_service'] for more info


Regardless of whether you use the `verbose` or `raise_warnings` options in the `validate()` method call, any problems identified can be accessed through the `ModelValidator.warnings` attribute.

In [4]:
pp.pprint(model_validator.warnings)

{'mismatched_node_names': [(15, 'Albertas', 'Alberta'),
                           (288, 'Furnace', 'Furnaces')],
 'nodes_no_provided_service': [(4, 'pyCIMS')],
 'unreferenced_nodes': [(289,
                         'pyCIMS.Canada.Alberta.Residential.Buildings.Shell.Space '
                         'heating.Furnaces')],
 'unspecified_nodes': [(49,
                        'pyCIMS.Canada.Alberta.Residential.Buildings.Shells'),
                       (59,
                        'pyCIMS.Canada.Alberta.Residential.Buildings.Shells'),
                       (286,
                        'pyCIMS.Canada.Alberta.Residential.Buildings.Shell.Space '
                        'heating.Furnace')]}


Ideally the code above returned an empty dictionary. If not, the examples below should help explain what the `warnings` dictionary might contain. 

First off, the `warnings` dictionary can contain up to 4 keys (as of December 2019). Those four keys are: 
* `mismatched_node_names`: a node's name (from the Node col) does not match the last component of its branch. 
* `unspecified_nodes`: a node is referenced in another node's "service requested" row but the node has not been specified within the mdoel description. 
* `unreferenced_nodes`: a non-root node is specified in the model description but its services are not requested by another node. 
* `nodes_no_provided_service`: a node is specified but does not provide a service. 

### Node Name & Node Branch Mismatch
```
'mismatched_node_names': [(16, 'Albertas', 'Alberta')]
```

Each list item indicates a mismatched node and branch name. The tuple contains (1) the row in the Excel file where the mismatch has occurred, (2) the name given to the node in the "Node" column, and (3) the name of the node according to the "Service provided" branch structure. 

### Unspecified Nodes
```
'unspecified_nodes': [(49, 'pyCIMS.Canada.Alberta.Residential.Buildings.Shells'),
                      (59, 'pyCIMS.Canada.Alberta.Residential.Buildings.Shells'),
                      (286, 'pyCIMS.Canada.Alberta.Residential.Buildings.Shell.Space heating.Furnace')]
```

Each list item indicates a service being requested from a node that was never specified in the model description. The tuple contains (1) the row in the Excel file where the reference is made, and (2) the node from which a service is being requested. 

### Unreferenced Nodes
```
'unreferenced_nodes': [(289, 'pyCIMS.Canada.Alberta.Residential.Buildings.Shell.Space heating.Furnaces')]
```

Each list item indicates a node specified in the model description but not requested by another node. The tuple contains (1) the row in the Excel file where the node was specified and (2) the name of the node (branch form). 

### Nodes that don't Provide a Service
```
 'nodes_no_provided_service': [(4, 'pyCIMS')]
```
Each list item indicates a node specified in the model description which does not provide a service. The associated tuple contains (1) the row in the Excel file where the node was specified and (2) the name of the node. 

# All the Code
Below, I've grouped together all the code needed for validating the model description. 

In [13]:
import os
import sys
import pprint as pp

sys.path.insert(0, os.path.abspath('../../')) # Assumes ipynb ran from pycims_prototype/code/model_validation
sys.path.insert(0, os.path.abspath('../../pyCIMS')) # Assumes ipynb ran from pycims_prototype/code/model_validation

import pyCIMS

model_description_file = '../../pyCIMS_model_description.xlsm' # Assumes ipynb ran from pycims_prototype/code/model_validation

model_validator = pyCIMS.ModelValidator(model_description_file)
model_validator.validate(verbose=False, raise_warnings=False)
print("Problems\n********")
pp.pprint(model_validator.warnings)

Problems
********
{'mismatched_node_names': [(15, 'Albertas', 'Alberta'),
                           (288, 'Furnace', 'Furnaces')],
 'nodes_no_provided_service': [(4, 'pyCIMS')],
 'unreferenced_nodes': [(289,
                         'pyCIMS.Canada.Alberta.Residential.Buildings.Shell.Space '
                         'heating.Furnaces')],
 'unspecified_nodes': [(49,
                        'pyCIMS.Canada.Alberta.Residential.Buildings.Shells'),
                       (59,
                        'pyCIMS.Canada.Alberta.Residential.Buildings.Shells'),
                       (286,
                        'pyCIMS.Canada.Alberta.Residential.Buildings.Shell.Space '
                        'heating.Furnace')]}
